# Explicación de los resultados

<h1 style="color:#1f77b4;"> Accuracy / F1 / Precision / Recall vs nº de clientes</h1>

In [5]:
import pandas as pd
from pathlib import Path
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# =========================
# CONFIG
# =========================
EXPERIMENTS = {
    "Balanced":      Path("experimentos_balanced"),
    "Dirichlet 0.5": Path("experimentos_DirichletPartioner_0.5"),
    "Dirichlet 1.0": Path("experimentos_DirichletPartioner_1.0"),
    "FeatureSkew":   Path("experimentos_FeatureSkew"),
}

DATASETS = {
    "HeartDisease": "HeartDisease",
    "Diabetes": "Diabetes",
    "BreastCancer": "breastcancer",
}

CLIENTS = [2, 4, 6]

METRICS = {
    "Accuracy": {
        "LocalTree Global": "acc_localTree_globalTest",
        "SuperTree Global": "acc_superTree_globalTest",
        "LocalTree Local":  "acc_localTree_localTest",
        "SuperTree Local":  "acc_superTree_localTest",
    },
    "F1": {
        "LocalTree Global": "f1_localTree_globalTest",
        "SuperTree Global": "f1_superTree_globalTest",
        "LocalTree Local":  "f1_localTree_localTest",
        "SuperTree Local":  "f1_superTree_localTest",
    },
    "Precision": {
        "LocalTree Global": "prec_localTree_globalTest",
        "SuperTree Global": "prec_superTree_globalTest",
        "LocalTree Local":  "prec_localTree_localTest",
        "SuperTree Local":  "prec_superTree_localTest",
    },
    "Recall": {
        "LocalTree Global": "rec_localTree_globalTest",
        "SuperTree Global": "rec_superTree_globalTest",
        "LocalTree Local":  "rec_localTree_localTest",
        "SuperTree Local":  "rec_superTree_localTest",
    },
}

SERIES_ORDER = ["LocalTree Global", "SuperTree Global", "LocalTree Local", "SuperTree Local"]

# 🔑 Colores fijos por serie (para que no cambie entre subplots)
SERIES_COLOR = {
    "LocalTree Global": "#1f77b4",
    "SuperTree Global": "#ff7f0e",
    "LocalTree Local":  "#2ca02c",
    "SuperTree Local":  "#d62728",
}

# =========================
# HELPERS
# =========================
def load_map(csv_path: Path):
    df = pd.read_csv(csv_path)
    return dict(zip(df["metric"], df["mean"]))

def find_csv(base: Path, n_clients: int):
    for name in [f"{n_clients}_Clients_Mean_global.csv", f"{n_clients}_Mean_global.csv"]:
        p = base / name
        if p.exists():
            return p
    return None

# =========================
# PLOTLY PLOTS
# =========================
for dataset_name, dataset_folder in DATASETS.items():

    n_rows = len(METRICS)
    n_cols = len(CLIENTS)

    # Subplot titles: solo primera fila (para que no se repitan)
    subplot_titles = []
    for r in range(n_rows):
        for c in range(n_cols):
            subplot_titles.append(f"{CLIENTS[c]} clientes" if r == 0 else "")

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        shared_yaxes=True,
        subplot_titles=subplot_titles,
        vertical_spacing=0.08,
        horizontal_spacing=0.06
    )

    # Orden fijo de experimentos en el eje X (aunque falte alguno)
    exp_order = list(EXPERIMENTS.keys())

    for col, n_clients in enumerate(CLIENTS, start=1):

        # cargar datos por experimento
        exp_data = {}
        for exp_name, exp_root in EXPERIMENTS.items():
            csv = find_csv(exp_root / dataset_folder, n_clients)
            if csv is None:
                continue
            exp_data[exp_name] = load_map(csv)

        # si no hay variedad de experimentos, saltamos
        if len(exp_data) < 2:
            continue

        # Usamos el orden fijo, pero solo pintamos los que existan (para no meter NaNs raros)
        exp_names = [e for e in exp_order if e in exp_data]

        for row, (metric_name, series_map) in enumerate(METRICS.items(), start=1):

            for label in SERIES_ORDER:
                key = series_map[label]
                vals = [exp_data[e].get(key, np.nan) for e in exp_names]

                customdata = [
                    [dataset_name, n_clients, metric_name, exp, label, v]
                    for exp, v in zip(exp_names, vals)
                ]

                fig.add_trace(
                    go.Bar(
                        x=exp_names,
                        y=vals,
                        name=label,
                        legendgroup=label,
                        showlegend=(row == 1 and col == 1),  # leyenda solo una vez
                        marker=dict(color=SERIES_COLOR[label]),  # 🔑 color fijo
                        customdata=customdata,
                        hovertemplate=(
                            "Dataset: %{customdata[0]}<br>"
                            "Clientes: %{customdata[1]}<br>"
                            "Métrica: %{customdata[2]}<br>"
                            "Experimento: %{customdata[3]}<br>"
                            "Serie: %{customdata[4]}<br>"
                            "Valor: %{customdata[5]:.4f}<extra></extra>"
                        ),
                    ),
                    row=row,
                    col=col
                )

            # títulos de eje Y por fila (métrica)
            if col == 1:
                fig.update_yaxes(title_text=metric_name, row=row, col=col)

            # etiqueta X solo en última fila
            if row == n_rows:
                fig.update_xaxes(title_text="Experimento", row=row, col=col)
            else:
                fig.update_xaxes(showticklabels=True, row=row, col=col)

    fig.update_layout(
        title=f"{dataset_name} — Métricas (Global vs Local) vs Experimento",
        barmode="group",
        bargap=0.25,
        height=900,
        width=1200,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
        margin=dict(t=90, l=70, r=30, b=60),
    )

    fig.show()

    # Guardar interactivo (opcional):
    # fig.write_html(f"{dataset_name}_metrics_global_vs_local.html", include_plotlyjs="cdn")


# RESULTADOS

## 🔹 HeartDisease

### Test LOCAL (verde vs rojo)
- **LocalTree Local** y **SuperTree Local** obtienen valores claramente más altos que en el test global.
- **SuperTree Local** suele ser ligeramente superior o igual a **LocalTree Local**  
  → la fusión no disminuye el rendimiento en el cliente.

**Por qué ocurre**  
En el test local, ambos modelos se evalúan sobre la misma distribución del cliente.  
El **LocalTree** está optimizado para ese cliente y el **SuperTree** incluye sus reglas locales dentro de la fusión, por lo que no se pierde información relevante. Esto indica que el proceso de agregación conserva correctamente el conocimiento local.

---

### Test GLOBAL (azul vs naranja)
- **LocalTree Global** y **SuperTree Global** presentan rendimientos similares, aunque **LocalTree Global** debería ser siempre peor.
- En *FeatureSkew*, **SuperTree Global** se ve más penalizado que en escenarios *Balanced*.

**Por qué ocurre**  
El **LocalTree Global** generaliza mal porque solo ha sido entrenado con la distribución de un único cliente.  
El **SuperTree Global**, aunque agrega información de varios clientes, no logra mejorar significativamente en *HeartDisease* debido a la alta heterogeneidad entre clientes: las reglas aprendidas son distintas y no existe una estructura global dominante. 

En *FeatureSkew*, esta falta de variables comunes entre clientes acentúa aún más el problema.

---

### Dirichlet 0.5 / 1.0
- En test local, ambos modelos mantienen buen rendimiento.
- En test global, **SuperTree Global** pierde capacidad predictiva.

**Por qué ocurre**  
El particionado Dirichlet introduce diferencias adicionales entre clientes. Aunque las reglas locales siguen siendo válidas en su propio contexto, el modelo global combina patrones inconsistentes, lo que reduce su capacidad de generalización y estabilidad en el test global.

---

## 🔹 Diabetes

### Test LOCAL
- Diferencias pequeñas entre **LocalTree Local** y **SuperTree Local**.
- **LocalTree Local** suele ser ligeramente superior,  
  pero **SuperTree Local** mantiene valores muy cercanos → buena transferencia.

**Por qué ocurre**  
Los clientes presentan distribuciones más similares que en *HeartDisease*, por lo que las reglas locales son compatibles entre sí. El SuperTree conserva estas reglas sin degradar el rendimiento local, mostrando una buena transferencia del conocimiento agregado.

---

### Test GLOBAL
- **SuperTree Global** es generalmente igual o mejor que **LocalTree Global**.
- Rendimiento estable al aumentar el número de clientes.

**Por qué ocurre**  
En *Diabetes* existen patrones comunes entre clientes. El SuperTree puede agregarlos de forma efectiva, mientras que el LocalTree Global sigue limitado por haber sido entrenado en una única distribución. Esto permite al SuperTree generalizar mejor a nivel global.

---

## 🔹 BreastCancer

### Test LOCAL
- **LocalTree Local** y **SuperTree Local** alcanzan rendimientos muy altos en todas las métricas.

**Por qué ocurre**  
El dataset presenta una estructura clara y estable, con variables dominantes compartidas entre clientes. Tanto los modelos locales como el agregado capturan fácilmente estas relaciones.

---

### Test GLOBAL
- **SuperTree Global ≥ LocalTree Global** de forma consistente.
- Resultados muy estables frente a cambios en partición y número de clientes.

**Por qué ocurre**  
Las reglas aprendidas por los distintos clientes son altamente consistentes. El SuperTree logra capturar una frontera de decisión global clara, mientras que el LocalTree Global sigue siendo más específico de un solo cliente. Esto hace que el modelo agregado generalice mejor y de forma más estable.


<h1 style="color:#5BD24B;"> Coverage / Complexity (Z) vs experimento</h1>


In [6]:
import pandas as pd
from pathlib import Path
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# =========================
# CONFIG
# =========================
EXPERIMENTS = {
    "Balanced":      Path("experimentos_balanced"),
    "Dirichlet 0.5": Path("experimentos_DirichletPartioner_0.5"),
    "Dirichlet 1.0": Path("experimentos_DirichletPartioner_1.0"),
    "FeatureSkew":   Path("experimentos_FeatureSkew"),
}

DATASETS = {
    "HeartDisease": "HeartDisease",
    "Diabetes": "Diabetes",
    "BreastCancer": "breastcancer",
}

CLIENTS = [2, 4, 6]  # columnas
METRICS_Z = {
    "FCC": ("fcc_local", "fcc_global"),
    "CMCA": ("cmca_local_global", None),      # solo una barra (local)
    "Silhouette": ("silhouette_local", "silhouette_global"),
}

# Orden fijo de experimentos en el eje X
EXP_ORDER = list(EXPERIMENTS.keys())

# Colores fijos (para que no cambien entre subplots)
COLOR_LOCAL = "#1f77b4"
COLOR_GLOBAL = "#ff7f0e"

# =========================
# HELPERS
# =========================
def load_map(csv_path: Path) -> dict:
    df = pd.read_csv(csv_path)
    return dict(zip(df["metric"], df["mean"]))

def find_csv(base_dir: Path, n: int):
    for name in [f"{n}_Clients_Mean_global.csv", f"{n}_Mean_global.csv"]:
        p = base_dir / name
        if p.exists():
            return p
    return None

# =========================
# PLOTLY PLOTS
# =========================
for dataset_name, dataset_folder in DATASETS.items():

    n_rows = len(METRICS_Z)
    n_cols = len(CLIENTS)

    # Títulos solo en la primera fila
    subplot_titles = []
    for r in range(n_rows):
        for c in range(n_cols):
            subplot_titles.append(f"{CLIENTS[c]} clientes" if r == 0 else "")

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        shared_yaxes=True,
        subplot_titles=subplot_titles,
        vertical_spacing=0.10,
        horizontal_spacing=0.06
    )

    for col, n_clients in enumerate(CLIENTS, start=1):

        # cargar experimentos disponibles
        exp_maps = {}
        for exp_name, exp_root in EXPERIMENTS.items():
            csv = find_csv(exp_root / dataset_folder, n_clients)
            if csv is None:
                continue
            exp_maps[exp_name] = load_map(csv)

        if len(exp_maps) < 1:
            continue

        # orden fijo pero solo los que existan
        exp_names = [e for e in EXP_ORDER if e in exp_maps]

        for row, (metric_name, (k_local, k_global)) in enumerate(METRICS_Z.items(), start=1):

            # --- Local siempre ---
            local_vals = [exp_maps[e].get(k_local, np.nan) for e in exp_names]
            local_custom = [
                [dataset_name, n_clients, metric_name, exp, "Local", v]
                for exp, v in zip(exp_names, local_vals)
            ]

            fig.add_trace(
                go.Bar(
                    x=exp_names,
                    y=local_vals,
                    name="Local",
                    legendgroup="Local",
                    showlegend=(row == 1 and col == 1),
                    marker=dict(color=COLOR_LOCAL),
                    customdata=local_custom,
                    hovertemplate=(
                        "Dataset: %{customdata[0]}<br>"
                        "Clientes: %{customdata[1]}<br>"
                        "Métrica: %{customdata[2]}<br>"
                        "Experimento: %{customdata[3]}<br>"
                        "Serie: %{customdata[4]}<br>"
                        "Valor: %{customdata[5]:.4f}<extra></extra>"
                    ),
                ),
                row=row,
                col=col
            )

            # --- Global solo si existe ---
            if k_global is not None:
                global_vals = [exp_maps[e].get(k_global, np.nan) for e in exp_names]
                global_custom = [
                    [dataset_name, n_clients, metric_name, exp, "Global", v]
                    for exp, v in zip(exp_names, global_vals)
                ]

                fig.add_trace(
                    go.Bar(
                        x=exp_names,
                        y=global_vals,
                        name="Global",
                        legendgroup="Global",
                        showlegend=(row == 1 and col == 1),
                        marker=dict(color=COLOR_GLOBAL),
                        customdata=global_custom,
                        hovertemplate=(
                            "Dataset: %{customdata[0]}<br>"
                            "Clientes: %{customdata[1]}<br>"
                            "Métrica: %{customdata[2]}<br>"
                            "Experimento: %{customdata[3]}<br>"
                            "Serie: %{customdata[4]}<br>"
                            "Valor: %{customdata[5]:.4f}<extra></extra>"
                        ),
                    ),
                    row=row,
                    col=col
                )

            # Y label por fila
            if col == 1:
                fig.update_yaxes(title_text=metric_name, row=row, col=col)

            # X label solo última fila
            if row == n_rows:
                fig.update_xaxes(title_text="Experimento", row=row, col=col)

    fig.update_layout(
        title=f"{dataset_name} — Métricas Z vs Experimento",
        barmode="group",
        bargap=0.25,
        height=850,
        width=1200,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
        margin=dict(t=90, l=70, r=30, b=60),
    )

    fig.show()

    # Guardar interactivo (opcional):
    # fig.write_html(f"{dataset_name}_Z_metrics_vs_experiment.html", include_plotlyjs="cdn")


## 🔹 FCC — Factual Coverage Coherence

**Qué mide**  
Mide la **coherencia en el espacio de variables** entre una explicación factual y sus explicaciones contrafactuales.

**Definición**

$$
\mathrm{FCC}(F, CF) =
\frac{|\mathrm{vars}(F) \cap \mathrm{vars}(CF)|}
     {|\mathrm{vars}(CF)|}
$$

**Donde**
- $\mathrm{vars}(F)$: conjunto de variables (*features*) que aparecen en la explicación factual $F$.
- $\mathrm{vars}(CF)$: conjunto de variables que aparecen en una explicación contrafactual $CF$.

**Interpretación**
- **FCC alto** → los contrafactuales utilizan las mismas variables que el factual  
  → explicación **internamente coherente**.
- **FCC bajo** → los contrafactuales introducen variables nuevas  
  → explicación **poco alineada o inconsistente**.

---

## 🔹 CMCA — Counterfactual Minimal Change Agreement

**Qué mide**  
Mide la **alineación entre explicaciones contrafactuales locales y globales**, en términos de variables utilizadas.

**Definición**

$$
\mathrm{CMCA} =
\frac{1}{|\mathrm{CF}_L|}
\sum_{cf \in \mathrm{CF}_L}
\max_{cf' \in \mathrm{CF}_G}
\frac{|\mathrm{vars}(cf) \cap \mathrm{vars}(cf')|}
     {|\mathrm{vars}(cf)|}
$$

**Donde**
- $\mathrm{CF}_L$: conjunto de contrafactuales locales.
- $\mathrm{CF}_G$: conjunto de contrafactuales globales.

**Interpretación**
- **CMCA alto** → los contrafactuales locales están **bien alineados** con los globales.
- **CMCA bajo** → las explicaciones locales y globales **utilizan variables distintas**.

---

## 🔹 Silhouette

**Qué mide**  
Evalúa la **calidad estructural de la vecindad sintética** generada para producir las explicaciones.

**Interpretación**
- **Silhouette positivo** → buena separación de la vecindad  
  → explicaciones **estables y fiables**.
- **Silhouette negativo** → vecindad mezclada  
  → explicaciones **poco fiables**.



-------------
------------
# RESULTADOS

## 🔹 HeartDisease

### FCC
- **Local > Global** de forma consistente.
- *FeatureSkew* destaca: el modelo local mantiene mucha coherencia interna.
- El modelo global pierde FCC con más clientes → mezcla de reglas.

### CMCA
- Valores **moderados–bajos**.
- Ligera mejora con más clientes, pero persiste la **desalineación local–global**.
- Indica que el modelo global no reutiliza bien las mismas variables explicativas.

### Silhouette
- Muy sensible al particionado.
- *Dirichlet* genera valores negativos → vecindad poco fiable.
- *FeatureSkew* mejora la estabilidad local, pero el global sigue inestable.

---

## 🔹 Diabetes

### FCC
- Muy alto en **Local**, especialmente con **2 clientes**.
- El modelo global mejora al aumentar clientes, pero sigue por debajo.

### CMCA
- Mejor comportamiento que en *HeartDisease*.
- Con **4–6 clientes** hay mejor alineación local–global.

### Silhouette
- Bastante problemática:
  - El modelo global es muy negativo en varios escenarios.
  - *Dirichlet 1.0* es el peor caso.
- Indica vecindades sintéticas poco separables.

---

## 🔹 BreastCancer

### FCC
- **Global > Local** en casi todos los casos.
- Muy estable incluso con *FeatureSkew*.
- Indica reglas globales consistentes.

### CMCA
- Valores bajos pero estables.
- Mejoran ligeramente con más clientes.

### Silhouette
- El modelo global es **claramente positivo**.
- El modelo local es frecuentemente negativo → sobreajuste local.
- Dataset favorable a explicaciones globales.